<a href="https://colab.research.google.com/github/SWazniewicz/UMwF-1-/blob/main/Zadanie_3_UMwF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stwórz model generujący sygnały kupna i sprzedaży na rynku
Decyzje ma generować jeden z algorytmów uczenia maszynowego, po optymalizacji hiperparametrów
Zrób backtest wybranego modelu, użyj biblioteki przystosowanej do backtestów
Pamiętaj o wnioskach i wizualizacji wyników
4 pkt z oceny będą zależne od wyników inwestycji
Inwestujesz w wylosowaną spółkę przez określony okres, ale można wykorzystać także inne dane niż historyczne (np. obliczone wskaźniki)

Strategia inwestycyjna (decyzje wejścia i wyjścia) dla spółki  Microsoft (MSFT), test w okresie od 01.01.2024 - 06.05.2024


In [ ]:
!pip install yfinance backtesting scikit-learn ta --quiet

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import ta

In [ ]:
symbol = "MSFT"
start_train = "2020-01-01"
end_all = "2024-05-06"
test_start = "2024-01-01"
test_end = "2024-05-06"

data = yf.download(symbol, start=start_train, end=end_all)
data.head()

In [ ]:
print(data.columns)

In [ ]:
data.columns = data.columns.get_level_values(0)

In [ ]:
print(data.columns)
print(data.head())

In [ ]:
df = data.copy()

# Stopy zwrotu
df["return_1d"] = df["Close"].pct_change()
df["return_5d"] = df["Close"].pct_change(5)
df["return_10d"] = df["Close"].pct_change(10)

# Średnie kroczące
df["sma_10"] = df["Close"].rolling(10).mean()
df["sma_20"] = df["Close"].rolling(20).mean()
df["sma_50"] = df["Close"].rolling(50).mean()

# RSI
df["rsi_14"] = ta.momentum.rsi(df["Close"], window=14)

# MACD
macd = ta.trend.MACD(df["Close"])
df["macd"] = macd.macd()
df["macd_signal"] = macd.macd_signal()
df["macd_diff"] = macd.macd_diff()

# Wolumen
df["vol_sma_10"] = df["Volume"].rolling(10).mean()
df["vol_sma_20"] = df["Volume"].rolling(20).mean()

df.dropna(inplace=True)
df.head()

In [ ]:
df["future_return_1d"] = df["Close"].pct_change().shift(-1)
df["target"] = (df["future_return_1d"] > 0).astype(int)
df.dropna(inplace=True)

In [ ]:
test_start = "2024-01-01"
test_end = "2024-05-06"

feature_cols = [
    "return_1d", "return_5d", "return_10d",
    "sma_10", "sma_20", "sma_50",
    "rsi_14",
    "macd", "macd_signal", "macd_diff",
    "vol_sma_10", "vol_sma_20"
]

X = df[feature_cols]
y = df["target"]

train_mask = df.index < test_start
test_mask = (df.index >= test_start) & (df.index <= test_end)

X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[test_mask], y[test_mask]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from scipy.stats import randint

tscv = TimeSeriesSplit(n_splits=5)

rf = RandomForestClassifier(random_state=42, n_jobs=-1)

param_distributions = {
    "n_estimators": randint(50, 300),
    "max_depth": randint(3, 20),
    "min_samples_split": randint(2, 20),
    "min_samples_leaf": randint(1, 10),
    "max_features": ["sqrt", "log2", None]
}

search = RandomizedSearchCV(
    rf,
    param_distributions=param_distributions,
    n_iter=30,
    scoring="accuracy",
    cv=tscv,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

search.fit(X_train, y_train)
best_model = search.best_estimator_
print("Najlepsze parametry:", search.best_params_)

In [ ]:
df["signal_proba"] = best_model.predict_proba(X)[:, 1]
df["signal"] = (df["signal_proba"] > 0.55).astype(int)

In [ ]:
from backtesting import Backtest, Strategy


bt_data = df.copy()
bt_data = bt_data[["Open", "High", "Low", "Close", "Volume", "signal"]]
bt_data_test = bt_data.loc[test_start:test_end].copy()

In [ ]:
from backtesting import Strategy

class MLSignalStrategy(Strategy):
    def init(self):
        pass

    def next(self):
        signal = self.data.signal[-1]

        # Jeśli sygnał = 1 → chcemy być w pozycji long
        if signal == 1:
            if not self.position:
                self.buy()

        # Jeśli sygnał = 0 → wychodzimy z pozycji
        else:
            if self.position:
                self.position.close()

In [ ]:
bt = Backtest(
    bt_data_test,
    MLSignalStrategy,
    cash=10000,
    commission=0.001,
    exclusive_orders=True
)

stats = bt.run()
stats

Backtest strategii został przeprowadzony przy użyciu biblioteki backtesting.py, z kapitałem początkowym 10 000 USD oraz prowizją transakcyjną 0.1% od każdej transakcji. Zastosowano założenie jednej otwartej pozycji w danym czasie. Strategia osiągnęła dodatnią stopę zwrotu na poziomie 3.25% w analizowanym okresie, przy bardzo niskim maksymalnym obsunięciu kapitału wynoszącym 1.32%. Wysoki współczynnik Sharpe’a (2.06) wskazuje na korzystny stosunek zysku do ryzyka. W porównaniu do strategii „kup i trzymaj”, model osiągnął niższą stopę zwrotu, jednak charakteryzował się znacznie mniejszą ekspozycją na rynek oraz niższą zmiennością portfela.

In [ ]:
bt.plot()

Wizualizacja backtestu potwierdza konserwatywny charakter strategii opartej na modelu uczenia maszynowego. Strategia generowała niewielką liczbę transakcji, unikała długotrwałych spadków cen oraz charakteryzowała się niskim obsunięciem kapitału.